## Import Data


In [3]:
import pandas as pd
file = pd.read_csv("hdac.txt", sep = "\t")
### Get only IC50 that has equal acticit and the units of nM
query = file.query('STANDARD_TYPE == "IC50" and RELATION == "=" and STANDARD_UNITS == "nM"')

IC50 = query['STANDARD_VALUE']
SMILES = query['CANONICAL_SMILES']


In [4]:
query.count(axis=0)

CMPD_CHEMBLID            707
MOLREGNO                 707
PARENT_CMPD_CHEMBLID     707
PARENT_MOLREGNO          707
MOL_PREF_NAME            131
COMPOUND_KEY             707
MOLWEIGHT                700
ALOGP                    697
PSA                      697
NUM_RO5_VIOLATIONS       697
CANONICAL_SMILES         707
ACTIVITY_ID              707
STANDARD_TYPE            707
RELATION                 707
STANDARD_VALUE           707
STANDARD_UNITS           707
PCHEMBL_VALUE            701
ACTIVITY_COMMENT           0
DATA_VALIDITY_COMMENT      6
POTENTIAL_DUPLICATE      707
BAO_ENDPOINT             707
UO_UNITS                 707
QUDT_UNITS               707
ASSAY_ID                 707
ASSAY_CHEMBLID           707
ASSAY_TYPE               707
DESCRIPTION              707
ASSAY_SRC_ID             707
ASSAY_SRC_DESCRIPTION    707
ASSAY_ORGANISM           567
ASSAY_STRAIN               0
ASSAY_TAX_ID             567
CURATED_BY               707
BAO_FORMAT               707
TID           

## Standardized molecules

In [5]:
from rdkit.Chem.SaltRemover import SaltRemover
from rdkit.Chem import MolFromSmiles,MolToSmiles

remover = SaltRemover()
len(remover.salts)

SMILES_desalt = []

for i in SMILES:
    mol = MolFromSmiles(i) 
    mol_desalt = remover.StripMol(mol)
    mol_SMILES = MolToSmiles(mol_desalt)
    SMILES_desalt.append(mol_SMILES)

ImportError: No module named 'rdkit'

In [6]:
from rdkit import *

ImportError: No module named 'rdkit'